In [2]:
import sys

In [3]:
import networkx as nx

import pandas as pd

import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (20.0, 10.0)

import requests
url = 'https://raw.githubusercontent.com/RauVR/TF_GRUPO03-COMPLEJIDAD/main/pqdict.py'
r= requests.get(url)

with open('pqdict.py','w')as f:
  f.write(r.text)

from pqdict import PQDict

from collections import defaultdict
import ast

In [4]:
vuelos_eeuu = pd.read_csv('https://raw.githubusercontent.com/RauVR/TF_GRUPO03-COMPLEJIDAD/main/vuelos.csv')
vuelos_eeuu.head()

,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DISTANCE,FLY_DATE,LATITUDE,LONGITUDE
0,ANC,SEA,1448,1/10/2008,61.17432,-149.99619
1,LAX,PBI,2330,1/11/1990,33.94254,-118.40807
2,SFO,CLT,2296,1/12/1990,37.61900,-122.37484
3,LAX,MIA,2342,1/10/1990,33.94254,-118.40807
4,SEA,ANC,1448,1/02/1990,47.44898,-122.30931


In [6]:
origen = {}
destino = {}
distancia = {}
i=0

for row in vuelos_eeuu.iterrows():
  origen[i]= row[1]["ORIGIN_AIRPORT"]
  destino[i]= row[1]["DESTINATION_AIRPORT"]
  distancia[i]= row[1]["DISTANCE"]
  i= i +1

visitados=[]

Graph={}

for i in range(len(origen)):
  if origen[i] not in visitados:
    visitados.append(origen[i])
    a = "{"
    for j in range(len(origen)):
      if origen[i]==origen[j]:
        a = a + "'"+destino[j] + "' :" + str(distancia[j]) + ","
    a = a +"}"
    Graph[origen[i]] = ast.literal_eval(a)

DG=nx.DiGraph()
for row in vuelos_eeuu.iterrows():
  DG.add_edge(row[1]["ORIGIN_AIRPORT"],
              row[1]["DESTINATION_AIRPORT"],
              weight=row[1]["DISTANCE"]
              )

###**ALGORITMO DIJSKTRA**

In [7]:
def dijkstra(G, start, end=None):
  print("Origen: "+start)
  print("Destino: "+ end)
  inf = float('inf')
  D = {start: 0}
  Q = PQDict(D)
  P = {}
  U = set(G.keys())
  while U:
      (v, d) = Q.popitem()
      D[v] = d
      U.remove(v)
      if v == end: break

      for w in G[v]:
          if w in U:
              d = D[v] + G[v][w]
              if d < Q.get(w, inf):
                  Q[w] = d
                  P[w] = v
  print("DISTANCIA:",d)
  return D, P


###**PARA ENCONTRAR CAMINO MÁS CORTO**

In [9]:
def shortest_path(G, start, end):
    dist, pred = dijkstra(G, start, end)
    v = end
    path = [v]
    while v != start:
        v = pred[v]
        path.append(v)
    path.reverse()
    return path

#--CAMINO CORTO ENTRE 2 PUNTOS--
print(shortest_path(Graph, "LAS", "FLL"))

Origen: LAS
Destino: FLL
DISTANCIA: 2174
['LAS', 'DFW', 'FLL']


##**OBSTENCIÓN DEL CAMINO MÍNIMO**

In [12]:
def plot_shortest_path(path):
    print(path)
    positions = nx.random_layout(DG)

    nx.draw(DG, pos=positions,
                node_color='lightblue',
                edge_color='gray',
                font_size=10,
                width=1, with_labels=True, node_size=1500, alpha=0.8
           )

    short_path=nx.DiGraph()
    for i in range(len(path)-1):
        short_path.add_edge(path[i], path[i+1])

    nx.draw(short_path, pos=positions,
                node_color='dodgerblue',
                edge_color='dodgerblue',
                font_size=10,
                width=3, with_labels=True, node_size=1500
           )

    plt.show()

##**MUESTRA DEL GRAFO**

In [11]:
#Muestreo en gráfico del camino más corto
camino=shortest_path(Graph, "LAS", "FLL")
print("Camino más corto: ")
plot_shortest_path(camino)

Output hidden; open in https://colab.research.google.com to view.